Реализовать разложение Гаусса $ PA = LU $ (с выбором ведущего элемента), использовать его для нахождения обратной матрицы $ A^{-1} $. Организовать проверку, вычислив $ A \^{A^{-1}} $, где $ \^{A^{-1}} $ --- найденная обратная матрица.

In [8]:
import numpy as np

def LU_decomposition(A):
    """
    LU-разложение матрицы A с выбором ведущего элемента.

    Args:
        A: Квадратная матрица.

    Returns:
        P: Матрица перестановок.
        L: Нижнетреугольная матрица.
        U: Верхнетреугольная матрица.
    """
    n = A.shape[0]

    L = np.eye(n, dtype=float)
    U = A.copy()
    P = np.eye(n, dtype=float)

    for i in range(n - 1):
        leader_idx = np.argmax(np.abs(U[i:, i])) + i

        if leader_idx != i:
            P[[i, leader_idx]] = P[[leader_idx, i]]
            U[[i, leader_idx]] = U[[leader_idx, i]]
            if i > 0:
                L[[i, leader_idx], :i] = L[[leader_idx, i], :i]

        for j in range(i + 1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, :] = U[j, :] - L[j, i] * U[i, :]

    return P, L, U


def solve_LU(L, U, P, b):
    """
    Решение системы Ax = b с использованием LU-разложения.

    Args:
        L: Нижняя треугольная матрица.
        U: Верхняя треугольная матрица.
        P: Матрица перестановок.
        b: Вектор правой части.

    Returns:
        x: Решение системы Ax = b.
    """

    # Решаем Ly = Pb
    Pb = P @ b
    y = np.zeros_like(Pb)
    for i in range(len(Pb)):
        y[i] = Pb[i] - np.dot(L[i, :i], y[:i])

    # Решаем Ux = y
    x = np.zeros_like(y)
    for i in range(len(y) - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]

    return x

def inverse_A(A):
    """
    Нахождение обратной матрицы A.

    Args:
        A: Квадратная матрица.

    Returns:
        A_inv: Обратная матрица A.
    """
    n = A.shape[0]
    P, L, U = LU_decomposition(A)

    E = np.eye(n, dtype=float)

    A_inv = np.zeros((n, n))
    for i in range(n):
        b = E[:, i]
        x = solve_LU(L, U, P, b)
        A_inv[:, i] = x

    return A_inv

In [12]:
A = np.array([[2, 1, 1],
              [4, 3, 3],
              [8, 7, 9]], dtype=float)

print("A = \n", A)
print("\n")

P, L, U = LU_decomposition(A)
A_inv = inverse_A(A)

print("A_inv = \n", A_inv)
print("\n")

print("===CHECK===")

E = A @ A_inv
print("A * A_inv = \n", E)

A = 
 [[2. 1. 1.]
 [4. 3. 3.]
 [8. 7. 9.]]


A_inv = 
 [[ 1.5 -0.5  0. ]
 [-3.   2.5 -0.5]
 [ 1.  -1.5  0.5]]


===CHECK===
A * A_inv = 
 [[1.00000000e+00 2.22044605e-16 0.00000000e+00]
 [5.55111512e-16 1.00000000e+00 0.00000000e+00]
 [7.77156117e-16 2.22044605e-16 1.00000000e+00]]
